#Importando bibliotecas

In [ ]:
import pandas as pd
import sqlite3
import io

#Abertura dos arquivos

In [ ]:
#Abro o csv e gero um dataframe com suas informações
df = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/source/20230424-tabelapanespecies.csv', encoding='unicode_escape', sep=';')

#Renomeio alguma colunas
df = df.rename(
    columns={
        "idTaxon": "IDEspecie",
        "taxonNome": "Nome_Cientifico",
    }
)

#Removo a coluna 315, que tinha uma espécie apagada
df = df.drop(315)

#Removo a coluna pan_nome, que é redundante
df = df.drop(columns=['pan_nome'])

#Corrijo os nomes de 534 e 856
df = df.replace({'Crypturellus noctivagus noctivagus Crypturellus noctivagus':'Crypturellus noctivagus noctivagus'})

#Quebro a coluna nome científico em 3 nomes
df[['Nome_genero', 'Epiteto_especifico', 'subespecie']] = df['Nome_Cientifico'].str.split(expand=True)

#Mudo a ordem das colunas
df = df.iloc[:, [2,0,1,3,4,5]]

#Possíveis mudanças, excluir IDEspecie, que parece inútil, excluir nome_cientifico

df

#Com problemas
# 315, não é mais subespécie
# 534 e 856, Crypturellus noctivagus noctivagus Crypturellus noctivagus

,idPan,IDEspecie,Nome_Cientifico,Nome_genero,Epiteto_especifico,subespecie
0,CEMAVE_27.02,10488,Cyanopsitta spixii,Cyanopsitta,spixii,None
1,RAN_10.02,7915,Adelophryne maranguapensis,Adelophryne,maranguapensis,None
2,RAN_10.02,7935,Hylomantis granulosa,Hylomantis,granulosa,None
3,RAN_10.02,12115,Glaucomastix abaetensis,Glaucomastix,abaetensis,None
4,RAN_10.02,12121,Ameivula nativo,Ameivula,nativo,None
...,...,...,...,...,...,...
2391,CEPTA_09.01,6333,Bunocephalus larai,Bunocephalus,larai,None
2392,CEPTA_09.01,4391,Characidium oiticicai,Characidium,oiticicai,None
2393,CEPTA_09.01,7368,Pseudopimelodus mangurus,Pseudopimelodus,mangurus,None
2394,CEPTA_09.01,7191,Rhinelepis aspera,Rhinelepis,aspera,None


#Colocando dados no sql e criando .dump

In [ ]:
#Crio a database e a conexão
conn = sqlite3.connect('test_database')
c = conn.cursor()

#Crio o esquema para a tabela
c.execute('CREATE TABLE IF NOT EXISTS especies (id_expecie number, nome_cientifico text, idPan text, pan_nome text)')
conn.commit()

#Passo as informações do dataframe pandas para a tabela sql
df.to_sql('especies', conn, if_exists='replace', index = False)

#Crio o dump em sql
with open('dump_mexido.sql', 'w') as f:
    for line in conn.iterdump():
        f.write('%s\n' % line)
conn.close()

#Contando valor de instâncias abrangência

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/FelipeGomide/TP2_IBD/main/source/20230424-tabelapandados.csv', encoding='ISO-8859-1', sep=';')

df = df.iloc[:, [4,5]]
df = df.drop_duplicates()
df = df.reset_index()
df

,index,panAbrangenciaTaxonomica,panAbrangenciaGeografica
0,0,Intraclasse,Nacional
1,2,Monoespecífico,Bioma
2,3,Multiclasse,Ecossistema
3,5,Monoespecífico,Específico (distribuição da espécie)
4,6,Intraclasse,Bioma
5,16,Intraclasse,Ecossistema
6,54,Multiclasse,Bacia
7,55,Multiclasse,Bioma
8,64,Multiclasse,Nacional
9,65,Intraclasse,Bacia
